### 一. 前期概念
1. Region是一个连续范围内的rowkey空间, 连续指Region中的rowkey在start key到end key范围内排序  
2. Region之间不会产生交叉, 一个rowkey只能属于一个Region, 被一个RegionServer管理
3. meta表中, 使用3层B树找到某个表的某个rowkey所在的Region   
4. 当初次创建一张表时, Hbase只会分配一个Region给这张表. 这意味着初始时, 所有的request都会通往一个RegionServer上

### 二. PRE-SPLITTING
1. 为什么使用预先划分?  
 上面讲到, 一张表初始创建时, 因为只有一个Region, 所以所有的请求都会发往一个RegionServer, 造成压力巨大. 如果创建表时就制定Region的数量, 并按照一定的规则将rowkey发往不同的Region, 就会在初始就使用集群的分布式功能
 
2. 预划分的start key和end key生成策略  
  预划分需要计算表的分割点"split points", 有两种默认的预划分算法: 
    * `HexStringSplit`:   
     如果rowkey是ASCII码表示的, 可以使通过MD5生成或者其他产生均匀分布的16进制表示法. rowkey范围为"00000000" => "FFFFFFFF"的范围, 左侧补0来使rowkey长度相同. 它把rowkey均匀分配到Region上, 这中做法会造成更大的空间使用量, 且不够直观
    * `UniformSplit`:   
    如果rowkey是字节数组,可以用UniformSplit的方式进行，按照原始byte值（从0x00~0xFF）右边以00填充。以这种方式分区的表在插入的时候需要对rowkey进行一个技巧性的改造， 比如原来的rowkey为rawStr，则需要对其取hashCode，然后进行按照比特位反转后放在最初rowkey串的前面
    ```shell
    Integer.reverse(Integer.valueOf(Integer.valueOf(i).hashCode())))
    ```
    
3. 通常使用预划分建表指定region数量以及region划分算法, 后面使用Region自动划分

### 三. AUTO SPLITTING
1. 什么时候执行自动划分   
   无论是否开启了pre-spliting, 一旦Region到一定限制, 就会自动化分为2个Region. 有3种预定义的自动划分策略
    * ConstantSizeRegionSplitPolicy
    * IncreasingToUpperBoundRegionSplitPolicy
    * KeyPrefixRegionSplitPolicy.